In [1]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from keras import backend as K

In [3]:
K.set_learning_phase(0)

In [10]:
from keras.models import load_model
import os
dirname = os.path.dirname(os.path.abspath("__file__"))
model = load_model(os.path.join(dirname, 'coreapi/dataset/keras_model.h5'))
print(model.outputs)

[<tf.Tensor 'activation_1/Softmax:0' shape=(?, 1683) dtype=float32>]


In [11]:
print(model.outputs)

[<tf.Tensor 'activation_1/Softmax:0' shape=(?, 1683) dtype=float32>]


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         336600    
_________________________________________________________________
lstm_5 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_3 (Dense)              (None, 1683)              338283    
_________________________________________________________________
activation_1 (Activation)    (None, 1683)              0         
Total params: 995,683
Trainable params: 995,683
Non-trainable params: 0
_________________________________________________________________


In [13]:
import os
MODEL_DIR = "/home/bhupinder/PycharmProjects/untitled1"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))
if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !sudo rm -r {export_path}

tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'inpput_img': model.input},
    outputs={'output': model.output})
!saved_model_cli show --dir {export_path} --all
os.environ["MODEL_DIR"] = MODEL_DIR

export_path = /home/bhupinder/PycharmProjects/untitled1/1


Already saved a model, cleaning up

[sudo] password for bhupinder: 

In [8]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=850 \
  --model_name=model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [9]:
!tail server.log

2019-01-02 23:42:02.462814: I tensorflow_serving/core/loader_harness.cc:66] Approving load for servable version {name: model version: 1}
2019-01-02 23:42:02.462851: I tensorflow_serving/core/loader_harness.cc:74] Loading servable version {name: model version: 1}
2019-01-02 23:42:02.462915: I external/org_tensorflow/tensorflow/contrib/session_bundle/bundle_shim.cc:363] Attempting to load native SavedModelBundle in bundle-shim from: /home/bhupinder/PycharmProjects/untitled1/1
2019-01-02 23:42:02.462944: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:31] Reading SavedModel from: /home/bhupinder/PycharmProjects/untitled1/1
2019-01-02 23:42:02.485172: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:54] Reading meta graph with tags { serve }
2019-01-02 23:42:02.502974: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-01-02 23:42:02.532819:

In [10]:
import requests
import json
data = json.dumps({"instances" : [[23]]})
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:850/v1/models/model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)
import numpy as np
np.asanyarray(predictions['predictions'])

array([[0.00010675, 0.00012183, 0.00012073, ..., 0.00010677, 0.0001071 ,
        0.0001069 ]])